In [2]:
from pathlib import Path
import pickle

import pandas as pd
import lightgbm as lgb

In [3]:
model_path = '../../../../models/lgbm_model.pkl'

valid_data_dir_path = '../../../../data/valid'
data_file_format = 'csv'
target_col = ['target']
objective = 'regression' # multiclass
num_class = '3'
valid_metrics = 'rmse'
learning_rate = 0.1
model_path = '../models/model.pkl'

In [20]:
# Dump the objects
Path(model_path).parent.mkdir(
    parents=True, exist_ok=True
) 
with open(model_path, 'wb') as f:
  pickle.dump(model, f)